In [1]:
import numpy as np
import pyscf
import matplotlib.pyplot as plt
import pyscf.lo


/Users/heather/opt/anaconda3/envs/pyqmc_dev/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Setting h5py.default_file_mode is deprecated. 'r' (read-only) is the default from h5py 3.0.
  h5py.get_config().default_file_mode = 'r'


In [2]:
mol = pyscf.gto.M(atom="H 0. 0. 0.; H 0. 0. 1.4",
                  unit='B',
                  basis='minao')
mf = pyscf.scf.RHF(mol)
mf.kernel()

converged SCF energy = -1.09056621169749


-1.090566211697487

In [3]:
def get_iao(mol, mf, mo_coeff, ref_bas = 'minao'):
  a = pyscf.lo.iao.iao(mol, mo_coeff, minao=ref_bas) 
  a = pyscf.lo.vec_lowdin(a, mf.get_ovlp())
  R_ao_lo = np.einsum('ji,jk->ik', a, mf.get_ovlp())
  return a, R_ao_lo

a, R_ao_lo = get_iao(mol, mf, mf.mo_coeff[:,:2])
R_mo_lo = np.einsum('ji,jk,kl->il',a,mf.get_ovlp(),mf.mo_coeff[:,:2])

In [4]:
# Obtain the 1e TB model Hamiltonian in LO
ham_tb_lo = np.einsum('i,ai,bi->ab', mf.mo_energy, R_mo_lo, R_mo_lo)
print('TB Ham:\n', ham_tb_lo)

# check that the TB model gives the same mo energies
eigvals, eigvecs = np.linalg.eigh(ham_tb_lo)
print('TB eigenvals:', eigvals)
print('HF mo energies:', mf.mo_energy)

TB Ham:
 [[-0.11020441 -0.50912646]
 [-0.50912646 -0.11020441]]
TB eigenvals: [-0.61933087  0.39892205]
HF mo energies: [-0.61933087  0.39892205]


In [16]:
# compute the 1e integrals
ham_1e_kin = pyscf.gto.getints("int1e_kin_sph", mol._atm, mol._bas, mol._env)
ham_1e_nuc = pyscf.gto.getints("int1e_nuc_sph", mol._atm, mol._bas, mol._env)

ham_1e = ham_1e_kin + ham_1e_nuc

# transform into MO basis
ham_1e_mo = np.einsum('kl,ki,lj', ham_1e, mf.mo_coeff, mf.mo_coeff)
print('direct 1e integrals in MO:\n', ham_1e_mo)


direct 1e integrals in MO:
 [[-1.18552106e+00 -1.11022302e-16]
 [ 0.00000000e+00 -5.73440850e-01]]


In [26]:
mf.energy_elec()[0] - mf.energy_elec()[1]

-2.371042114764098

In [17]:
# transform into the localized basis
ham_1e_lo = np.einsum('kl,ki,lj', ham_1e, a, a)
print('direct 1e integrals in LO:\n', ham_1e_lo)

direct 1e integrals in LO:
 [[-0.87948095 -0.3060401 ]
 [-0.3060401  -0.87948095]]


In [28]:
# compute the 2e integrals
int_2e_ao = pyscf.gto.getints("int2e_sph", mol._atm, mol._bas, mol._env)
int_2e_mo = np.einsum('klmn,ka,lb,mc,nd->abcd', int_2e_ao, mf.mo_coeff, mf.mo_coeff, mf.mo_coeff, mf.mo_coeff)

print('direct 2e integrals in MO:\n',int_2e_mo)

direct 2e integrals in MO:
 [[[[ 5.66190189e-01 -5.55111512e-17]
   [-5.55111512e-17  5.56277523e-01]]

  [[-2.77555756e-17  1.40192148e-01]
   [ 1.40192148e-01 -1.11022302e-16]]]


 [[[-1.38777878e-16  1.40192148e-01]
   [ 1.40192148e-01 -5.55111512e-16]]

  [[ 5.56277523e-01 -3.33066907e-16]
   [ 0.00000000e+00  5.85863852e-01]]]]


In [27]:
int_2e_lo = np.einsum('klmn,ka,lb,mc,nd->abcd', int_2e_ao, a, a, a, a)

print('direct 2e integrals in LO:\n',int_2e_lo)

direct 2e integrals in LO:
 [[[[ 0.70634442 -0.00491842]
   [-0.00491842  0.42596012]]

  [[-0.00491842  0.00987475]
   [ 0.00987475 -0.00491842]]]


 [[[-0.00491842  0.00987475]
   [ 0.00987475 -0.00491842]]

  [[ 0.42596012 -0.00491842]
   [-0.00491842  0.70634442]]]]
